In [2]:
%pip install torch --no-cache-dir
%pip install --upgrade pytorch-lightning
%pip install tensorboard
%pip install torchmetrics<0.7.0
%pip install torchvision
%pip install alive-progress
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/bin/sh: 1: cannot open 0.7.0: No such file
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import boto3
train_data = datasets.CIFAR10(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
print(train_data)
# split train data to train and validation
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = torch.utils.data.random_split(train_data, [train_size, val_size])

print(train_data)
print(val_data)
test_data = datasets.CIFAR10(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

from data_utils import CIFAR10_dataset
from sagemaker import get_execution_role
role = get_execution_role()
# N_COMPONENTS = 12*12
# train_data = CIFAR10_dataset(train = True, n_components = N_COMPONENTS)


In [4]:
train_data, val_data = torch.utils.data.random_split(train_data, [40000, 10000])

ValueError: Sum of input lengths does not equal the length of the input dataset!

In [ ]:
train_data.__getitem__(0)[0].permute(1,2,0).shape

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
ratio = 250/255
for i in range(20):
    ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[])
    ax.imshow(((train_data.__getitem__(i)[0].permute(1,2,0).numpy())/ratio).astype('uint8'), cmap=plt.cm.bone, interpolation='nearest')
plt.show()

In [ ]:
from torch.utils.data import DataLoader
loaders = {
    'train' : DataLoader(train_data, 
                                          batch_size=64, 
                                          shuffle=True, 
                                          num_workers=0),
    'valid' : DataLoader(val_data,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=0),
#     'test'  : DataLoader(test_data, 
#                                           batch_size=64, 
#                                           shuffle=True, 
#                                           num_workers=0),
}



In [6]:
from convolutional_nets import convnext_model
model = convnext_model(in_channels = 3, num_classes = 10, 
                        dropout = .2, learning_rate = 2e-4, image_size = 32)

Some weights of the model checkpoint at facebook/convnext-tiny-224 were not used when initializing ConvNextModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ConvNextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import torch
# set up GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [ ]:
# put model in train mode

model.train()
torch.set_grad_enabled(True)

train_losses = []
valid_losses = []
all_train_logs = []
all_valid_logs = []
total_batches = len(loaders['train'])
for epoch in range(3):
    batch_num = 0
    print("Epoch number: ", epoch)
    for train_batch, test_batch in zip(loaders['train'], loaders['valid']):
            # train step
            train_loss, train_logs = model.training_step(train_batch)
            valid_loss, valid_logs = model.validation_step(test_batch)
            if(batch_num % 10 == 0):
                print("On batch {} of epoch {}".format(batch_num, epoch)
                                 + "\n" + "Train accuracy: " + str(train_logs['train_acc'].numpy()) + "\n" + "Train loss: " + str(train_logs['train_loss'].detach().numpy())
                                 + "\n" + "Valid accuracy: " + str(valid_logs['val_acc'].numpy()) + "\n" + "Valid loss: " + str(valid_logs['val_loss'].detach().numpy()))
            batch_num+=1
            # clear gradients
            model.optimizer.zero_grad()

            # backward
            train_loss.backward()

            # update parameters
            model.optimizer.step()
            
            # append to losses
            train_losses.append(train_loss)
            all_train_logs.append(train_logs)
            valid_losses.append(valid_loss)
            all_valid_logs.append(valid_logs)

Epoch number:  0
[2022-04-26 22:13:07.082 1-8-1-cpu-py36-ml-m5-4xlarge-89e57170236a1e63cdf2d0ba6b56:3557 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-26 22:13:07.130 1-8-1-cpu-py36-ml-m5-4xlarge-89e57170236a1e63cdf2d0ba6b56:3557 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
On batch 0 of epoch 0
Train accuracy: 0.140625
Train loss: 2.2894473
Valid accuracy: 0.109375
Valid loss: 2.2911437
On batch 10 of epoch 0
Train accuracy: 0.375
Train loss: 2.1224556
Valid accuracy: 0.375
Valid loss: 2.1614878
On batch 20 of epoch 0
Train accuracy: 0.5
Train loss: 1.8101988
Valid accuracy: 0.53125
Valid loss: 1.81293
On batch 30 of epoch 0
Train accuracy: 0.5625
Train loss: 1.4931283
Valid accuracy: 0.640625
Valid loss: 1.4495425
On batch 40 of epoch 0
Train accuracy: 0.609375
Train loss: 1.2318554
Valid accuracy: 0.6875
Valid loss: 1.1520474
On batch 50 of epoch 0
Train accuracy: 0.75
Train loss: 1.0301

In [ ]:
# save pytorch model and logs
import numpy as np
torch.save(model.state_dict(), 'models/CIFAR10_model_{}x{}.pt'.format(np.sqrt(N_COMPONENTS).astype('uint8'),np.sqrt(N_COMPONENTS).astype('uint8')))
import pickle
with open('models/train_logs_CIFAR10_model_{}x{}.pkl'.format(np.sqrt(N_COMPONENTS).astype('uint8'),np.sqrt(N_COMPONENTS).astype('uint8')), 'wb') as f:
    pickle.dump(all_train_logs, f)
with open('models/val_logs_CIFAR10_model_{}x{}.pkl'.format(np.sqrt(N_COMPONENTS).astype('uint8'),np.sqrt(N_COMPONENTS).astype('uint8')), 'wb') as f:
    pickle.dump(all_valid_logs, f)